# Multiprocess with multiple parameters

There's an interesting limitation to the pool map functions that's explored in this notebook - out of the box they only accept one argument.


In [1]:
import multiprocess
import platform
import warnings

In [2]:
# shenanigans
os = platform.system()
if os == "Darwin":
    try:
        multiprocess.set_start_method("spawn")
    except RuntimeError:
        pass

In [3]:
def f(x, y):
    """raise the power of x to y"""
    return x**y


In [4]:
# uses Pool to raise 100 integers to the power of 2
p = multiprocess.Pool(multiprocess.cpu_count() - 1)
"""All alternative approaches below would raise error:
TypeError: f() missing 1 required positional argument: 'y'

"""
#result = p.map(f, range(10), 2)
#result = p.map(f, x=range(10), y=2)
#result = p.map(f, {"x":range(10), "y":2})
#result = p.map(f, [range(10), 2])
#result = p.map(f, [(range(10), 2))


#print(result.get(timeout=1))

"All alternative approaches below would raise error:\nTypeError: f() missing 1 required positional argument: 'y'\n\n"

In order to pass multiple arguments, the solution I've found is to use a mapper function.

The function `pool_process()` below was written originally for work on the data science campus' [access to services OTP repo](https://github.com/datasciencecampus/access_to_services_OTP/blob/dev/src/make_features/call_OTP_api.py).

It creates the pool and maps the function to the parameters.

Depending on the function you are attempting to map, you will need to create a specific parameter mapper. Some examples of how this has been implemented within the OTP repo:

[`pool_process()`](https://github.com/datasciencecampus/access_to_services_OTP/blob/dev/src/make_features/call_OTP_api.py#L734)  

[`retry_mapper()`](https://github.com/datasciencecampus/access_to_services_OTP/blob/dev/src/make_features/call_OTP_api.py#L785)  

[`pspace_wrapper`](https://github.com/datasciencecampus/access_to_services_OTP/blob/dev/src/make_features/call_OTP_api.py#L802)  

And their implementation [throughout this notebook](https://github.com/datasciencecampus/access_to_services_OTP/blob/dev/notebooks/scale_up/01-sample_isochrone_extent.ipynb).


[Here's a great blog on your options for multiple params](https://miguendes.me/how-to-pass-multiple-arguments-to-a-map-function-in-python)
